In [6]:
# C:\Users\yebel\Desktop\OPENCLASSROOMS\FORMATION_DATA_ANALYST\OPENCLASSROOMS_COURS\PROJET10
# PROJET10_DATA_ANALYSIS_OCR
# Changement du repertoire: A ne plus changer.
import os
os.chdir('C://Users//yebel//Desktop//OPENCLASSROOMS//FORMATION_DATA_ANALYST//OPENCLASSROOMS_COURS//PROJET10')

In [7]:
# 0- IMPORTATIONS DES LIBRAIRIES NECESSAIRES POUR L'ANALYSE DES DONNEES.

import sys
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from joblib import dump, load

## Le modèle de régression logistique avec la Cross-Validation.

In [8]:
# 1-Importation du modèle choisit pour l'analyse.

model1_reg_log_sklearn_CV = load("DATA/Logistic_Trained_model.joblib")

# Visualisation du modèle.

model1_reg_log_sklearn_CV

LogisticRegressionCV()

In [9]:
# 2-Importation des données de l'expérimentation.

#new_data_test = pd.read_csv('DATA/billets_production.csv', sep=";", decimal=".", header=0)

new_data_test = pd.read_csv('DATA/billets_test.csv', sep=";", decimal=".", header=0)

In [10]:
# Visualisation des données de l'expérimentation.

new_data_test

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5


In [11]:
# Vérifions que toutes les données sont correctes.

# vérification des variables.

liste_X = ["diagonal","height_left","height_right","margin_low","margin_up","length","id"]
if all([item in new_data_test.columns for item in liste_X]):
    print("Les variables du fichier sont correctes. Voici un aperçu des données :\n\n",new_data_test)
else :
    messagebox.showerror(title="Erreur d'importation du fichier", message="Attention, les colonnes du fichier "+root.filename+" ne correspondent pas à la trame attendue !")
    sys.exit()

Les variables du fichier sont correctes. Voici un aperçu des données :

    diagonal  height_left  height_right  margin_low  margin_up  length   id
0    172.09       103.95        103.73        4.39       3.09  113.19  B_1
1    171.52       104.17        104.03        5.27       3.16  111.82  B_2
2    171.78       103.80        103.75        3.81       3.24  113.39  B_3
3    172.02       104.08        103.99        5.57       3.30  111.10  B_4
4    171.79       104.34        104.37        5.00       3.07  111.87  B_5


In [12]:
# Vérification de l'absence des valeurs nulles dans les données de l'expérimentation.

if((new_data_test.isnull().sum(axis = 0)).sum() > 0):
    messagebox.showerror(title="Valeurs nulles détectées", message="Attention, des valeurs nulles ont été détectées dans le fichier ! Traitement impossible en l'état.")
    sys.exit()
else :
    print("Aucune valeur nulle détectée, le traitement peut continuer.")

Aucune valeur nulle détectée, le traitement peut continuer.


In [13]:
# Déterminons des variables explicatives du Target (de la variable objectif).

new_X = new_data_test[liste_X[:-1]]

# Visualisation de la matrice des variables explicatives.

new_X

,diagonal,height_left,height_right,margin_low,margin_up,length
0,172.09,103.95,103.73,4.39,3.09,113.19
1,171.52,104.17,104.03,5.27,3.16,111.82
2,171.78,103.80,103.75,3.81,3.24,113.39
3,172.02,104.08,103.99,5.57,3.30,111.10
4,171.79,104.34,104.37,5.00,3.07,111.87


In [14]:
# Déterminons les probabilités à affecter à l'échantillon de l'expérimentation.

new_proba = model1_reg_log_sklearn_CV.predict_proba(new_X)

In [15]:
# Déterminons le score de presence : Probalilité que le billet soit VRAI (is_genuine = 0).

new_score = new_proba[:,0]

In [16]:
# Intégration de la probabilité new_score dans le fichier d'expérimentation: new_data_test.

new_data_test["proba_true"] = new_score

In [17]:
#Détermination et définition de la Target variable (la variable objectif): is_genuine.

new_data_test["is_genuine"] = np.where(new_data_test["proba_true"]>=0.5, 'Oui', 'Non')

In [18]:
# Traitement et explication du Dataframe finale pour une meilleure perception et compréhension.

new_data_test_final = new_data_test[["id","proba_true","is_genuine"]].rename(columns={"id":"Identifiant du billet ayant été testé",
                                                                                      "proba_true":"Probabilité que le billet soit VRAI (%)",
                                                                                      "is_genuine":"Le billet est-il authentique ?"})  

# Arrondir la probabilité que le billet soit vrai à deux chiffres après la virgule.

new_data_test_final["Probabilité que le billet soit VRAI (%)"] = round((new_data_test_final["Probabilité que le billet soit VRAI (%)"]*100),2)

In [19]:
# Visualisation du fichier de l'expérimentation finale et interprétation des résultats.

new_data_test_final

,Identifiant du billet ayant été testé,Probabilité que le billet soit VRAI (%),Le billet est-il authentique ?
0,B_1,0.25,Non
1,B_2,99.86,Oui
2,B_3,0.01,Non
3,B_4,100.00,Oui
4,B_5,99.54,Oui


In [20]:
# Etablissons la liste des faux billets de notre expérimentation.

Liste_faux_billets = list(new_data_test_final.loc[(new_data_test_final["Le billet est-il authentique ?"] == "Non"),"Identifiant du billet ayant été testé"].values)

# Visualisation de la Liste des faux billets.

print("Les billets dont l'identifiant est dans la liste", Liste_faux_billets, "\nont été identifiés commes FAUX selon l'algorithme de prédiction.")

Les billets dont l'identifiant est dans la liste ['B_1', 'B_3'] 
ont été identifiés commes FAUX selon l'algorithme de prédiction.
